##### 저장된 모델 활용
- 모델 파일 종류
    * 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 가능
    * 모델 전체 저장 파일 => 바로 로딩 후 사용 가능


[1] 모듈 로딩 <hr>

In [ ]:
# 모듈 로딩

import sys
sys.path.append(r'C:\Users\KDP-50\OneDrive\바탕 화면\KDT_DYH\10.Natural_Language_Processing')

import pickle
import pandas as pd
import torch
from torchinfo import summary
from NLPmodules import funcRNN

ModuleNotFoundError: No module named 'NLP_Class'

In [ ]:
## 모델 파일 관련

# 저장 경로
SAVE_PATH='../Project/saved_models/'

# 모델 구조 및 파라미터 모두 저장 파일명
# SAVE_MODEL='model_score(0.7739)_loss(0.4754).pth' # 다 1 모델
SAVE_MODEL='hun_model_score(0.8598)_loss(0.3825).pth' # 반대로 모델
# SAVE_MODEL='model_score(0.9452)_loss(0.1809).pth' # 다 0 모델

[2] 모델 로딩 - 모델 전체 파일 사용

In [3]:
NLP_Model = torch.load(SAVE_PATH+SAVE_MODEL, weights_only=False)

In [4]:
print(NLP_Model)

SentenceClassifier(
  (embedding): Embedding(2695, 128, padding_idx=0)
  (model): LSTM(128, 64, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (classifier): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [5]:
summary(NLP_Model)

Layer (type:depth-idx)                   Param #
SentenceClassifier                       --
├─Embedding: 1-1                         344,960
├─LSTM: 1-2                              198,656
├─Linear: 1-3                            129
├─Dropout: 1-4                           --
Total params: 543,745
Trainable params: 543,745
Non-trainable params: 0

[3] 예측 <hr>

- vocab 로딩하기

In [6]:
# 저장된 vocab 파일 불러오기
with open('hun_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [7]:
type(vocab)

dict

In [8]:
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH=50

In [14]:
# 예시 (결과: 0)
data = "근육 괴사와 독소로 인한 전신증상을 가져오는 중증 질환이다"
prediction = funcRNN.predict_model(NLP_Model, data, vocab, MAX_LENGTH)
print("Prediction:", prediction)

Prediction: False


In [15]:
# 예시 (결과: 1)
data = "오늘날의 기준적 수술식은 적응과 요약에도 달려 있지만 일반적으로 경복막 제왕절술이다"
prediction = funcRNN.predict_model(NLP_Model, data, vocab, MAX_LENGTH)
print("Prediction:", prediction)

Prediction: True


In [12]:
# 데이터가 문자열이라면 토큰화 및 인덱스로 변환
tokens = [vocab.get(token, vocab['oov']) for token in data]  # 토큰을 인덱스로 변환

# 패딩하여 모델의 입력 형태와 일치시키기
if len(tokens) < MAX_LENGTH:
    tokens = tokens + [vocab['pad']] * (MAX_LENGTH - len(tokens))
else:
    tokens = tokens[:MAX_LENGTH]

dataTS = torch.LongTensor(tokens).unsqueeze(0)

# 검증 모드로 모델 설정
NLP_Model.eval()
with torch.no_grad():
    # 추론/평가
    logits = NLP_Model(dataTS)
    pre_val = torch.sigmoid(logits)

prediction = (pre_val >= 0.5).float()
print(prediction)

tensor([[1.]])
